### 1- Upload Libraries

In [276]:
import pyspark
from pyspark.sql import SparkSession

import pyspark.sql.functions as f

In [277]:
spark = SparkSession.builder.appName('Practise').getOrCreate()

### 2- Read Data

In [278]:
df = spark.read.csv('/Users/...', sep = '\t')

In [279]:
# Show only first 5 rows
df.show(5)

+--------+--------+--------------------+------------------+----------+
|     _c0|     _c1|                 _c2|               _c3|       _c4|
+--------+--------+--------------------+------------------+----------+
|order_id|quantity|           item_name|choice_description|item_price|
|       1|       1|Chips and Fresh T...|              NULL|    $2.39 |
|       1|       1|                Izze|      [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|           [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|              NULL|    $2.39 |
+--------+--------+--------------------+------------------+----------+
only showing top 5 rows



### 3-  Convert data headers original header

In [280]:
df = spark.read.option('header','true').csv('/Users/ilyasozgur/Desktop/Data_Science/Pandas/Data/Exercise1.csv', sep = '\t', inferSchema = True)

df.show(2)

+--------+--------+--------------------+------------------+----------+
|order_id|quantity|           item_name|choice_description|item_price|
+--------+--------+--------------------+------------------+----------+
|       1|       1|Chips and Fresh T...|              NULL|    $2.39 |
|       1|       1|                Izze|      [Clementine]|    $3.39 |
+--------+--------+--------------------+------------------+----------+
only showing top 2 rows



### 4- Total Number of Rows

In [281]:
# Total number of observation in the dataset
print((df.count(), len(df.columns)))

(4622, 5)


### 5- Total Number of Columns

In [282]:
# The number of columns of the dataset

len(df.columns)

5

### 6- Print Name of Columns

In [283]:
# Print the name of the columns 

df.columns 

['order_id', 'quantity', 'item_name', 'choice_description', 'item_price']

### 7-  Which was the most ordered item?

In [284]:
df.show(2)

+--------+--------+--------------------+------------------+----------+
|order_id|quantity|           item_name|choice_description|item_price|
+--------+--------+--------------------+------------------+----------+
|       1|       1|Chips and Fresh T...|              NULL|    $2.39 |
|       1|       1|                Izze|      [Clementine]|    $3.39 |
+--------+--------+--------------------+------------------+----------+
only showing top 2 rows



In [285]:
# Groupby 
df_most_ordered_item = df.groupBy('item_name').sum('quantity')

# Rename the columns 
df_most_ordered_item = df_most_ordered_item.withColumnRenamed('sum(quantity)','Total_Order')

# Sort By
df_most_ordered_item = df_most_ordered_item.orderBy('Total_Order', ascending=False)

df_most_ordered_item.show(1)

+------------+-----------+
|   item_name|Total_Order|
+------------+-----------+
|Chicken Bowl|        761|
+------------+-----------+
only showing top 1 row



### 8-  What is the most ordered item in choice description column?

In [286]:
# Groupby 
df_most_ordered_item_cd = df.groupBy('choice_description').sum('quantity')

# Rename the columns 
df_most_ordered_item_cd = df_most_ordered_item_cd.withColumnRenamed('sum(quantity)','Total_Order')

# Sort By
df_most_ordered_item_cd = df_most_ordered_item_cd.orderBy('Total_Order', ascending=False)

# Drop Null values
df_most_ordered_item_cd = df_most_ordered_item_cd.na.drop(how = 'any', subset=['choice_description'])

# Filter data Frame
df_most_ordered_item_cd = df_most_ordered_item_cd.filter(df_most_ordered_item_cd['choice_description'] != 'NULL')

df_most_ordered_item_cd.show(1)

+------------------+-----------+
|choice_description|Total_Order|
+------------------+-----------+
|       [Diet Coke]|        159|
+------------------+-----------+
only showing top 1 row



### 9- How many item ordered in total?

In [287]:
# Groupby 
df_item_count = df.groupBy().sum('quantity')
df_item_count.show()

+-------------+
|sum(quantity)|
+-------------+
|         4972|
+-------------+



In [288]:
# Take sum of the specific column

import pyspark.sql.functions as f   

df_item_count = df.select(f.sum('quantity')).collect()[0][0]
df_item_count

4972

### 10- Check Column DataType and Split the column content and Convert into Float

In [289]:
# Check data types for the dataset

df.dtypes

[('order_id', 'int'),
 ('quantity', 'int'),
 ('item_name', 'string'),
 ('choice_description', 'string'),
 ('item_price', 'string')]

In [290]:
df.show(2)

+--------+--------+--------------------+------------------+----------+
|order_id|quantity|           item_name|choice_description|item_price|
+--------+--------+--------------------+------------------+----------+
|       1|       1|Chips and Fresh T...|              NULL|    $2.39 |
|       1|       1|                Izze|      [Clementine]|    $3.39 |
+--------+--------+--------------------+------------------+----------+
only showing top 2 rows



In [291]:
# Trim the columns

from pyspark.sql.functions import trim

df_trimmed = df.withColumn("item_price", trim(df['item_price']))

df_trimmed.show(2)

+--------+--------+--------------------+------------------+----------+
|order_id|quantity|           item_name|choice_description|item_price|
+--------+--------+--------------------+------------------+----------+
|       1|       1|Chips and Fresh T...|              NULL|     $2.39|
|       1|       1|                Izze|      [Clementine]|     $3.39|
+--------+--------+--------------------+------------------+----------+
only showing top 2 rows



In [292]:
# Clean up the '$' sign from the item_price column, only numbers

from pyspark.sql.functions import regexp_replace

df_trimmed = df_trimmed.withColumn("item_price",regexp_replace("item_price","[^0-9a-zA-Z_\-.]+", ""))

df_trimmed.show(2)

+--------+--------+--------------------+------------------+----------+
|order_id|quantity|           item_name|choice_description|item_price|
+--------+--------+--------------------+------------------+----------+
|       1|       1|Chips and Fresh T...|              NULL|      2.39|
|       1|       1|                Izze|      [Clementine]|      3.39|
+--------+--------+--------------------+------------------+----------+
only showing top 2 rows



In [293]:
# Check column datatypes

df_trimmed.printSchema()
df_trimmed.show(2)

root
 |-- order_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price: string (nullable = true)

+--------+--------+--------------------+------------------+----------+
|order_id|quantity|           item_name|choice_description|item_price|
+--------+--------+--------------------+------------------+----------+
|       1|       1|Chips and Fresh T...|              NULL|      2.39|
|       1|       1|                Izze|      [Clementine]|      3.39|
+--------+--------+--------------------+------------------+----------+
only showing top 2 rows



In [295]:
# Change datatypes of the column

df_trimmed = df_trimmed.withColumn("item_price", df_trimmed['item_price'].cast('float'))

df_trimmed.printSchema()
df_trimmed.show(2)

root
 |-- order_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price: float (nullable = true)

+--------+--------+--------------------+------------------+----------+
|order_id|quantity|           item_name|choice_description|item_price|
+--------+--------+--------------------+------------------+----------+
|       1|       1|Chips and Fresh T...|              NULL|      2.39|
|       1|       1|                Izze|      [Clementine]|      3.39|
+--------+--------+--------------------+------------------+----------+
only showing top 2 rows

